<a href="https://colab.research.google.com/github/a-donat/InProgress/blob/main/Sudoku16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import *

In [ ]:
my_grid = [
 ['5', ' ', ' ', ' ',   ' ', '7', ' ', ' ',   ' ', ' ', '4', 'E',   ' ', ' ', '6', 'D'],
 [' ', ' ', '9', 'D',   '8', ' ', ' ', '4',   ' ', ' ', '3', '6',   ' ', 'E', '2', ' '],
 [' ', '2', '6', '3',   '5', 'G', ' ', ' ',   ' ', ' ', '1', 'B',   ' ', ' ', 'A', ' '],
 [' ', ' ', ' ', 'A',   ' ', ' ', ' ', 'D',   ' ', ' ', ' ', ' ',   ' ', 'F', '5', ' '],

 [' ', '8', 'B', '1',   ' ', '3', ' ', ' ',   'D', '5', ' ', 'G',   ' ', ' ', ' ', ' '],
 ['E', ' ', ' ', ' ',   ' ', ' ', ' ', ' ',   ' ', 'C', ' ', ' ',   '2', ' ', ' ', ' '],
 [' ', ' ', '4', ' ',   ' ', ' ', ' ', ' ',   ' ', ' ', 'A', 'F',   ' ', ' ', ' ', '1'],
 ['G', ' ', ' ', 'F',   ' ', ' ', 'A', ' ',   ' ', '2', ' ', ' ',   '9', ' ', ' ', '8'],

 [' ', ' ', '7', ' ',   ' ', ' ', 'E', 'A',   ' ', ' ', ' ', ' ',   ' ', ' ', ' ', '4'],
 ['B', ' ', ' ', '8',   ' ', ' ', 'F', ' ',   ' ', '6', '9', 'A',   '5', ' ', ' ', ' '],
 [' ', '9', ' ', ' ',   ' ', ' ', ' ', 'G',   'E', ' ', 'F', ' ',   '6', '1', '8', ' '],
 ['D', ' ', 'F', '5',   '7', '9', '6', ' ',   '4', ' ', ' ', '1',   ' ', 'A', 'E', 'B'],

 [' ', 'C', ' ', '2',   ' ', 'A', ' ', ' ',   ' ', ' ', 'G', '8',   ' ', '3', ' ', ' '],
 [' ', 'G', '3', ' ',   ' ', 'E', '4', ' ',   ' ', '1', ' ', ' ',   ' ', '5', ' ', ' '],
 ['7', ' ', ' ', ' ',   ' ', ' ', 'B', ' ',   '3', ' ', ' ', ' ',   ' ', ' ', ' ', '2'],
 [' ', ' ', ' ', ' ',   ' ', '5', 'G', ' ',   ' ', ' ', ' ', ' ',   'F', 'D', 'B', ' ']
]

In [ ]:
possibilities = list("123456789ABCDEFG")

class Cell:
  def __init__(self, row_ind: int, col_ind: int, value: Optional[str]=None):
    self.row_ind = row_ind
    self.col_ind = col_ind
    self.box_row = int(row_ind/4)
    self.box_col = int(col_ind/4)
    self.value = value
    self.possibilities = set(possibilities) if value is None else set([])
    self.impossibilities = set()
    #self.box = int(row_ind/4)*4 + int(col_ind/4)

  def set_value(self, new_value: str):
    if self.value is not None:
      print("warning, cell already set")
    if new_value not in self.possibilities:
      print("warning, new_value not in possibilities")
    print("setting cell %d, %d to %s" % (self.row_ind, self.col_ind, new_value))
    self.value = new_value
    self.possibilities = set([])

  def set_remainder_as_value(self):
    """assumes len(self.possibilities) == 1"""
    self.set_value(list(self.possibilities)[0])


class Group:
  def __init__(self, cells: List[Cell]):
    self.cells = cells
    self.possibilities_list = [p for cell in cells for p in cell.possibilities]
    self.possibilities_set = set(self.possibilities_list)
    self.values = set([cell.value for cell in cells if cell.value is not None])
    self.tallies = {
        p: self.possibilities_list.count(p) for p in self.possibilities_set}
    self.possibility_locations = {
        p: [(cell.row_ind, cell.col_ind) for cell in cells if p in cell.possibilities]
        for p in self.possibilities_set}


class BoxGroup(Group):
  def __init__(self, cells, gridsize=16):
    super().__init__(cells)
    self.gridsize = gridsize
    self.box_rows = set([cell.row_ind for cell in cells])
    self.box_cols = set([cell.col_ind for cell in cells])
    self.possibility_rows = {
        p: set([cell.row_ind for cell in cells if p in cell.possibilities])
        for p in self.possibilities_set}
    self.possibility_cols = {
        p: set([cell.col_ind for cell in cells if p in cell.possibilities])
        for p in self.possibilities_set}
    self.removal_by_row = {
        p: [(list(rows)[0], c) for c in range(self.gridsize)
        if c not in self.box_cols
            ]
        for p, rows in self.possibility_rows.items() if len(rows) == 1}
    self.removal_by_col = {
        p: [(r, list(cols)[0]) for r in range(self.gridsize)
        if r not in self.box_rows
            ]
        for p, cols in self.possibility_cols.items() if len(cols) == 1}


class Grid:
  def __init__(self, cells: List[List[Cell]], size=16):
    self.cells = cells
    self.size = size
    self.box_size = int(size**.5)

  def Row(self, row: int):
    return Group(self.cells[row])

  def Col(self, col: int):
    return Group([row[col] for row in self.cells])

  def Box(self, box_row: int, box_col: int):
    return BoxGroup([
        self.cells[r][c]
        for r in range(box_row * self.box_size, (box_row + 1) * self.box_size)
        for c in range(box_col * self.box_size, (box_col + 1) * self.box_size)])

  def remove_cell_value_from_adjacent(self, cell):
    """ASSUMES ONLY 1 POSSIBILITY"""
    #print("CELL LOCATION IS [%d][%d]" % (cell.row_ind, cell.col_ind))
    for x in range(self.size):
      self.cells[cell.row_ind][x].possibilities -= set([cell.value])
      self.cells[x][cell.col_ind].possibilities -= set([cell.value])
    for rb in range(self.box_size):
      for cb in range(self.box_size):
        ri = int(cell.row_ind/4) * 4 + rb
        ci = int(cell.col_ind/4) * 4 + cb
        #print(".........removing %s from cell [%d][%d]" % (cell.value, ri, ci))
        self.cells[ri][ci].possibilities -= set([cell.value])

  def remove_rcb_values_from_cell(self, row, col):
    if self.cells[row][col].value is None:
      if len(self.cells[row][col].possibilities) == 1:
        self.cells[row][col].set_remainder_as_value()
      else:
        self.cells[row][col].possibilities -= self.Row(row).values
        self.cells[row][col].possibilities -= self.Col(col).values
        self.cells[row][col].possibilities -= self.Box(
            int(row/4), int(col/4)).values
        if len(self.cells[row][col].possibilities) == 1:
          self.cells[row][col].set_remainder_as_value()


  def remove_all_rcb_values(self):
    for row in range(self.size):
      for col in range(self.size):
        self.remove_rcb_values_from_cell(row, col)

  def set_only_possibility_in_group(self):
    for r in range(self.size):
      for p, l in self.Row(r).possibility_locations.items():
        if len(l) == 1:
          #print("row", r)
          self.cells[l[0][0]][l[0][1]].set_value(p)
          self.remove_cell_value_from_adjacent(self.cells[l[0][0]][l[0][1]])
      self.remove_all_rcb_values()
    for c in range(self.size):
      for p, l in self.Col(c).possibility_locations.items():
        if len(l) == 1:
          #print("col", c)
          self.cells[l[0][0]][l[0][1]].set_value(p)
          self.remove_cell_value_from_adjacent(self.cells[l[0][0]][l[0][1]])
      self.remove_all_rcb_values()
    for rb in range(4):
      for cb in range(4):
        for p, l in game.Box(rb, cb).possibility_locations.items():
          if len(l) == 1:
            #print("box", rb, cb)
            game.cells[l[0][0]][l[0][1]].set_value(p)
            self.remove_all_rcb_values()
          self.remove_all_rcb_values()
        self.remove_all_rcb_values()
      self.remove_all_rcb_values()

def create_blank_grid():
  grid = []
  for row in range(16):
    row = []
    for col in range(16):
      row.append(Cell(row, col))
    grid.append(row)
  return Grid(grid)

def create_grid_from_lists(grid_list: List[List[str]], nrows=16, ncols=16):
  grid = []
  for r in range(nrows):
    row = []
    for c in range(ncols):
      val = grid_list[r][c] if grid_list[r][c] != " " else None
      row.append(Cell(r, c, val))
    grid.append(row)
  return Grid(grid)


def create_box(first_row, first_col, values):
  cells = []
  for i in range(4):
    row = []
    for j in range(4):
      val = values[i][j] if values[i][j] != "x" else None
      #row.append(Cell(first_row + i, first_col + j, val))
      cells.append(Cell(first_row + i, first_col + j, val))
    #cells.append(row)
  return cells

In [ ]:
game = create_grid_from_lists(my_grid)

In [ ]:
game.remove_all_rcb_values()

setting cell 11, 1 to 3


In [ ]:
game.set_only_possibility_in_group()

setting cell 1, 8 to 5
setting cell 2, 9 to D
setting cell 2, 7 to E
setting cell 8, 14 to F
setting cell 12, 2 to 5
setting cell 6, 0 to 3
setting cell 0, 4 to A
setting cell 5, 15 to 5
setting cell 1, 9 to A
setting cell 4, 0 to 2
setting cell 4, 15 to F
setting cell 10, 6 to 5
setting cell 8, 13 to 9
setting cell 13, 3 to B


In [ ]:
game.set_only_possibility_in_group()

setting cell 4, 12 to A
setting cell 9, 13 to 2
setting cell 6, 12 to E
setting cell 10, 2 to 2


In [ ]:
game.set_only_possibility_in_group()

setting cell 10, 0 to A
setting cell 8, 12 to D
setting cell 10, 11 to D


In [ ]:
game.set_only_possibility_in_group()

In [ ]:
game.Box(0, 0).removal_by_col

{'7': [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1)],
 'B': [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1)]}

In [ ]:
game.cells[5][1].possibilities

{'3', '5', '6', '7', 'A', 'D'}

In [ ]:
for p, l in game.Row(1).possibility_locations.items():
  if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
      game.remove_cell_value_from_adjacent(game.cells[l[0][0]][l[0][1]])
  game.remove_all_rcb_values()

setting cell 1, 8 to 5
CELL LOCATION IS [1][8]


In [ ]:
for p, l in game.Row(2).possibility_locations.items():
  if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
      game.remove_cell_value_from_adjacent(game.cells[l[0][0]][l[0][1]])
  game.remove_all_rcb_values()

setting cell 2, 9 to D
setting cell 2, 7 to E


In [ ]:
for p, l in game.Row(8).possibility_locations.items():
  if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
      game.remove_cell_value_from_adjacent(game.cells[l[0][0]][l[0][1]])
  game.remove_all_rcb_values()

setting cell 8, 14 to F


In [ ]:
for p, l in game.Row(12).possibility_locations.items():
  if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
      game.remove_cell_value_from_adjacent(game.cells[l[0][0]][l[0][1]])
  game.remove_all_rcb_values()

setting cell 12, 2 to 5


In [ ]:
int(12/4), int(2/4)

(3, 0)

In [ ]:
for p, l in game.Col(0).possibility_locations.items():
  if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
      game.remove_cell_value_from_adjacent(game.cells[l[0][0]][l[0][1]])
  game.remove_all_rcb_values()

setting cell 6, 0 to 3


In [ ]:
game.Col(1).possibility_locations

{'D': [(5, 1), (6, 1), (7, 1), (14, 1)],
 '7': [(1, 1), (3, 1), (5, 1), (6, 1), (7, 1)],
 'B': [(0, 1), (1, 1), (3, 1)],
 '1': [(0, 1), (1, 1), (3, 1), (8, 1), (9, 1), (14, 1), (15, 1)],
 '4': [(3, 1), (9, 1), (14, 1), (15, 1)],
 'F': [(0, 1), (1, 1), (14, 1)],
 'A': [(5, 1), (14, 1), (15, 1)],
 '5': [(7, 1)],
 '6': [(5, 1), (6, 1), (7, 1), (8, 1), (14, 1), (15, 1)],
 'E': [(9, 1), (14, 1), (15, 1)]}

In [ ]:
game.Col(1).possibility_locations

{'5': [(7, 1)],
 'A': [(5, 1), (14, 1), (15, 1)],
 'D': [(5, 1), (6, 1), (7, 1), (14, 1)],
 '7': [(1, 1), (3, 1), (5, 1), (6, 1), (7, 1)],
 '1': [(0, 1), (1, 1), (3, 1), (8, 1), (9, 1), (14, 1), (15, 1)],
 'B': [(0, 1), (1, 1), (3, 1)],
 '6': [(5, 1), (6, 1), (7, 1), (8, 1), (14, 1), (15, 1)],
 'E': [(9, 1), (14, 1), (15, 1)],
 'F': [(0, 1), (1, 1), (14, 1)],
 '4': [(3, 1), (9, 1), (14, 1), (15, 1)]}

In [ ]:
game.cells[5][1].possibilities

{'6', '7', 'A', 'D'}

In [ ]:
game.cells[6][1].possibilities

{'6', '7', 'D'}

In [ ]:
game.set_only_possibility_in_group()

row 1
setting cell 1, 8 to 5
CELL LOCATION IS [1][8]
row 2
setting cell 2, 9 to D
CELL LOCATION IS [2][9]
row 2
setting cell 2, 7 to E
CELL LOCATION IS [2][7]
row 8
setting cell 8, 14 to F
CELL LOCATION IS [8][14]
row 12
setting cell 12, 2 to 5
CELL LOCATION IS [12][2]
col 0
setting cell 6, 0 to 3
CELL LOCATION IS [6][0]
col 4
setting cell 0, 4 to A
CELL LOCATION IS [0][4]
col 15
setting cell 5, 15 to 5
CELL LOCATION IS [5][15]
box 0 2
setting cell 1, 9 to A
box 1 0
setting cell 4, 0 to 2
box 1 3
setting cell 4, 15 to F
box 2 1
setting cell 10, 6 to 5
box 2 3
setting cell 8, 13 to 9
box 3 0
setting cell 13, 3 to B


In [ ]:
game.set_only_possibility_in_group()

row 4
setting cell 4, 12 to A
CELL LOCATION IS [4][12]
col 13
setting cell 9, 13 to 2
CELL LOCATION IS [9][13]
box 1 3
setting cell 6, 12 to E
box 2 0
setting cell 10, 2 to 2


In [ ]:
game.set_only_possibility_in_group()

setting cell 10, 0 to A
setting cell 5, 2 to D
setting cell 7, 2 to C
setting cell 14, 1 to D
setting cell 8, 12 to D
setting cell 10, 11 to D
setting cell 13, 10 to D


In [ ]:
x x x A
- x x x
x x A x
x A x x

x A x x
X X - -
! X X X
X X - -

In [ ]:
for r in range(16):
  for p, l in game.Row(r).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 1, 8 to 5
setting cell 2, 9 to D
setting cell 2, 7 to E
setting cell 8, 14 to F
-----------------


In [ ]:
for c in range(16):
  for p, l in game.Col(c).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 6, 0 to 3
setting cell 0, 4 to A
setting cell 5, 15 to 5
-----------------


In [ ]:
for rb in range(4):
  for cb in range(4):
    for p, l in game.Box(rb, cb).possibility_locations.items():
      if len(l) == 1:
        game.cells[l[0][0]][l[0][1]].set_value(p)
    print("-----------------")
    game.remove_all_rcb_values()
    print("-----------------")

-----------------
-----------------
-----------------
-----------------
A [(1, 9)]
setting cell 1, 9 to A
-----------------
-----------------
-----------------
-----------------
2 [(4, 0)]
setting cell 4, 0 to 2
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
F [(4, 15)]
setting cell 4, 15 to F
-----------------
-----------------
-----------------
-----------------
5 [(10, 6)]
setting cell 10, 6 to 5
-----------------
-----------------
-----------------
-----------------
9 [(8, 13)]
setting cell 8, 13 to 9
-----------------
-----------------
B [(13, 3)]
setting cell 13, 3 to B
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------


In [ ]:
for r in range(16):
  for p, l in game.Row(r).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 4, 12 to A
setting cell 12, 2 to 5
-----------------


In [ ]:
for c in range(16):
  for p, l in game.Col(c).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 9, 13 to 2
-----------------


In [ ]:
for rb in range(4):
  for cb in range(4):
    for p, l in game.Box(rb, cb).possibility_locations.items():
      if len(l) == 1:
        game.cells[l[0][0]][l[0][1]].set_value(p)
    print("-----------------")
    game.remove_all_rcb_values()
    print("-----------------")

-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
setting cell 6, 12 to E
-----------------
-----------------
setting cell 10, 2 to 2
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------


In [ ]:
for r in range(16):
  for p, l in game.Row(r).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 10, 0 to A
-----------------


In [ ]:
for c in range(16):
  for p, l in game.Col(c).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

setting cell 8, 12 to D
-----------------


In [ ]:
for rb in range(4):
  for cb in range(4):
    for p, l in game.Box(rb, cb).possibility_locations.items():
      if len(l) == 1:
        game.cells[l[0][0]][l[0][1]].set_value(p)
    print("-----------------")
    game.remove_all_rcb_values()
    print("-----------------")

-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
setting cell 10, 11 to D
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------


In [ ]:
for r in range(16):
  for p, l in game.Row(r).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

-----------------


In [ ]:
for c in range(16):
  for p, l in game.Col(c).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)
print("-----------------")
game.remove_all_rcb_values()

-----------------


In [ ]:
for rb in range(4):
  for cb in range(4):
    for p, l in game.Box(rb, cb).possibility_locations.items():
      if len(l) == 1:
        game.cells[l[0][0]][l[0][1]].set_value(p)
    print("-----------------")
    game.remove_all_rcb_values()
    print("-----------------")

-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------
-----------------


In [ ]:
game.Box(0, 0).possibility_locations

{'B': [(0, 1), (1, 1), (3, 1)],
 'C': [(0, 2), (0, 3), (1, 0), (2, 0), (3, 0), (3, 2)],
 '8': [(0, 2), (2, 0), (3, 0), (3, 2)],
 'G': [(0, 2), (0, 3), (3, 2)],
 '7': [(1, 1), (3, 1)],
 'E': [(3, 1), (3, 2)],
 'F': [(0, 1), (1, 0), (1, 1), (2, 0)],
 '1': [(0, 1), (0, 2), (1, 0), (1, 1), (3, 0), (3, 1), (3, 2)],
 '4': [(2, 0), (3, 0), (3, 1)]}

In [ ]:
{k: set([p[0] for p in v]) for k, v in game.Box(0, 0).possibility_locations.items()}

{'B': {0, 1, 3},
 'C': {0, 1, 2, 3},
 '8': {0, 2, 3},
 'G': {0, 3},
 '7': {1, 3},
 'E': {3},
 'F': {0, 1, 2},
 '1': {0, 1, 3},
 '4': {2, 3}}

In [ ]:
{k: set([p[1] for p in v]) for k, v in game.Box(0, 0).possibility_locations.items()}

{'B': {1},
 'C': {0, 2, 3},
 '8': {0, 2},
 'G': {2, 3},
 '7': {1},
 'E': {1, 2},
 'F': {0, 1},
 '1': {0, 1, 2},
 '4': {0, 1}}

In [ ]:
set([1,2,3]) - set([2])

{1, 3}

In [ ]:
{k: set([p[1] for p in v]) for k, v in game.Box(0, 3).possibility_locations.items()}


{'B': {12, 13},
 'C': {12, 13, 15},
 '3': {12, 15},
 '8': {12, 13},
 'G': {12, 13, 15},
 '7': {12, 13, 15},
 '9': {15},
 '1': {12},
 '4': {12, 13}}

In [ ]:
Group([game.cells[r][1] for r in range(16) if not r in [0, 1, 2, 3]]).possibilities_set

{'1', '4', '5', '6', '7', 'A', 'D', 'E', 'F'}

In [ ]:
{k: set([p[0] for p in v]) for k, v in game.Box(0, 1).possibility_locations.items()}

{'B': {0, 1, 3},
 'C': {0, 1, 2, 3},
 '3': {0, 3},
 '6': {3},
 '2': {0, 3},
 '9': {0, 2, 3},
 'F': {0, 1},
 '1': {0, 1, 3}}

In [ ]:
{k: set([p[0] for p in v]) for k, v in game.Box(2, 2).possibility_locations.items()}

{'B': {8, 10},
 'C': {8, 9, 11},
 '3': {8, 10},
 '8': {8, 11},
 'G': {8, 9, 11},
 '7': {9, 10},
 '5': {8},
 '2': {8, 11}}

In [ ]:
game.Box(0, 1).possibility_locations

{'B': [(0, 7), (1, 5), (3, 4), (3, 5)],
 'C': [(0, 6), (0, 7), (1, 5), (1, 6), (2, 6), (3, 4), (3, 5), (3, 6)],
 '3': [(0, 6), (0, 7), (3, 4), (3, 6)],
 'E': [(3, 4)],
 '6': [(3, 4), (3, 5)],
 '2': [(0, 6), (0, 7), (3, 4), (3, 5), (3, 6)],
 '9': [(0, 6), (0, 7), (2, 6), (3, 4), (3, 6)],
 'F': [(0, 7), (1, 5)],
 '1': [(0, 6), (0, 7), (1, 5), (1, 6), (3, 4), (3, 5), (3, 6)]}

In [ ]:
game.Box(0, 1).possibilities_set

{'1', '2', '3', '6', '9', 'B', 'C', 'E', 'F'}

In [ ]:
game.Box(0, 1).values

{'4', '5', '7', '8', 'A', 'D', 'E', 'G'}

In [ ]:
game.remove_all_rcb_values()

In [ ]:
for r in range(16):
  for p, l in game.Row(r).possibility_locations.items():
    if len(l) == 1:
      game.cells[l[0][0]][l[0][1]].set_value(p)

setting cell 1, 9 to A
setting cell 3, 5 to 6
setting cell 7, 10 to E
setting cell 12, 2 to 5


In [ ]:
game.Row(3).possibilities_set

{'1', '2', '3', '4', '6', '7', '8', '9', 'B', 'C', 'G'}

In [ ]:
game.Row(3).possibility_locations

{'B': [(3, 1), (3, 5), (3, 12)],
 'C': [(3, 0),
  (3, 2),
  (3, 5),
  (3, 6),
  (3, 8),
  (3, 10),
  (3, 11),
  (3, 12),
  (3, 15)],
 '3': [(3, 6), (3, 12), (3, 15)],
 '8': [(3, 0), (3, 2), (3, 8), (3, 9), (3, 10), (3, 12)],
 '7': [(3, 1), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 15)],
 'G': [(3, 2), (3, 8), (3, 9), (3, 12), (3, 15)],
 '2': [(3, 5), (3, 6), (3, 8), (3, 10), (3, 11)],
 '9': [(3, 6), (3, 8), (3, 9), (3, 11), (3, 15)],
 '6': [(3, 5)],
 '1': [(3, 0), (3, 1), (3, 2), (3, 5), (3, 6), (3, 12)],
 '4': [(3, 0), (3, 1), (3, 12)]}

In [ ]:
game.cells[3][5].possibilities

{'1', '2', '6', 'B', 'C'}

In [ ]:
game.Box(
            int(3/4), int(5/4)).values

{'4', '5', '7', '8', 'A', 'D', 'E', 'G'}

In [ ]:
int(3/4), int(5/4)

(0, 1)

In [ ]:
game.Row(3).values

{'5', 'A', 'D', 'E', 'F'}

In [ ]:
game.Col(5).values

{'3', '5', '7', '9', 'A', 'E', 'G'}

In [ ]:
game.cells[1][8].set_value("5")

In [ ]:
game.Box(0, 2).possibility_locations

{'A': [(0, 8), (0, 9), (1, 8), (1, 9)],
 'D': [(2, 9)],
 'C': [(0, 8), (1, 8), (2, 8), (3, 8), (3, 10), (3, 11)],
 '8': [(0, 8), (0, 9), (2, 8), (2, 9), (3, 8), (3, 9), (3, 10)],
 'G': [(0, 8), (0, 9), (1, 8), (1, 9), (3, 8), (3, 9)],
 '7': [(1, 8), (1, 9), (2, 8), (2, 9), (3, 8), (3, 9), (3, 10), (3, 11)],
 '5': [(1, 8)],
 '2': [(0, 8), (3, 8), (3, 10), (3, 11)],
 '9': [(0, 8), (0, 9), (2, 8), (2, 9), (3, 8), (3, 9), (3, 11)],
 'F': [(0, 8), (0, 9), (1, 8), (1, 9), (2, 8), (2, 9)]}

In [ ]:
game.cells[8][1].possibilities

{'1', '3', '6'}

In [ ]:
game.Row(0).values_list

['5', '7', '4', 'E', '6', 'D']

In [ ]:
game.cells[0][1].possibilities -= game.Row(0).values()

In [ ]:
game.cells[0][1].possibilities

{'1', '2', '3', '8', '9', 'A', 'B', 'C', 'F', 'G'}

In [ ]:
game.cells[0][1].possibilities -= game.Col(1).values()

In [ ]:
game.cells[0][1].possibilities

{'1', '3', 'A', 'B', 'F'}

In [ ]:
game.cells[0][1].possibilities -= game.Box(0, 0).values()

In [ ]:
game.cells[0][1].possibilities

{'1', 'B', 'F'}

In [ ]:
from math import comb

In [ ]:
comb(7,2) + comb(7,3) + comb(7,4) + comb(7,5)

112

In [ ]:
set([1,3]) == set([3,1])

True

In [ ]:
my_grid = [[" "]*16]*16

In [ ]:
def display_grid(grid):
  for i in range(16):
    if i % 4 == 0:
      print("----------------------------------------")
    row = my_grid[i]
    print("|", ",".join(row[:4]),
          "|", ",".join(row[4:8]),
          "|", ",".join(row[8:12]),
          "|", ",".join(row[12:16]), "|")
  print("----------------------------------------")

In [ ]:
my_grid[0][0] = "5"
my_grid[0][5] = "7"
my_grid[0][10:12] = ["4", "E"]
my_grid[0][14:16] = ["6", "D"]


In [ ]:
display_grid(my_grid)

----------------------------------------
| 5, , ,  |  ,7, ,  |  , ,4,E |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
----------------------------------------
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
----------------------------------------
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
----------------------------------------
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
|  , , ,  |  , , ,  |  , , ,  |  , , ,  |
----------------------------------------


In [ ]:
my_grid

[['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' ',
  ' ',
  ' '],
 ['5',
  ' ',
  ' ',
  ' ',
  ' ',
  '7',
  ' ',
  ' ',
  ' ',
  ' ',
  '4',
  'E',
  ' ',
  ' '